In [ ]:
import sys
import subprocess
import os
# Get the current working directory
current_folder = os.getcwd()
modules_path = os.path.join(current_folder, 'modules')
sys.path.append(current_folder)
sys.path.append(modules_path)

restart_command = [sys.executable] + sys.argv

# Call subprocess to restart the Python interpreter
subprocess.call(restart_command)

import logging
import pandas as pd

from modules.api_yf import api_yf
from modules.db_sqlLite_utils import db_utils
from modules.plotter import pl
from modules.processdata import processdata

# Define the ETF ticker symbol
ticker = 'META'

# # Get a list of loaded modules
# loaded_modules = sys.modules.keys()
# # Print the list of loaded modules
# print(loaded_modules)


In [ ]:
data = db_utils.get_data_from_db(ticker)

data2 = processdata.add_daily_return_to_df(data)

start_date = processdata.formatDateDt(2021, 1, 1)
end_date = processdata.formatDateDt(2023, 1, 1)

# Calculate the return using the static method
total_return = processdata.get_return_defined_time(data2, start_date, end_date)


start_date = processdata.formatDateDt(2021, 1, 1)
end_date = processdata.formatDateDt(2024, 1, 1)

dStart, dEnd = processdata.get_valid_dates(data2, start_date, end_date)
roi = processdata.get_roi_defined_time(data2, dStart, dEnd)
mean = processdata.get_mean_between_dates(data2, dStart, dEnd)
std = processdata.get_sd_between_dates(data2, dStart, dEnd)
cov = processdata.get_cov_between_dates(data2, dStart, dEnd)

faang_list = ["META", "AMZN", "AAPL", "NFLX", "GOOG"]
mult_df = processdata.merge_df_by_column_name('daily_return',  '2020-1-1', '2020-12-31', *faang_list)

# Generate a Correlation Matrix
mult_df.corr()

# # We can look at the correlation between Netflix and the others
# mult_df.corr()['NFLX']

# # # We can plot this in a bar chart
mult_df.corr()['NFLX'].plot(kind='bar')

mult_df['NFLX'].var()

# Annualize by getting the number of samples and multiply
days = len(mult_df.index) # 253

nflx_a_var = mult_df['NFLX'].var() * 253
nflx_a_var

mult_df.cov() * 253

In [ ]:
# get data from yf
ticker = 'GOOG'

data = api_yf.getFullDataFromYf(ticker)
# insert data into sqlLite db
db_utils.execute_values(data, 'stock_data', ticker)

# retrive data from db
data = db_utils.get_data_from_db(ticker)
print(data.head())

# # Plot the closing prices over time using the Date column as x-axis
pl.plot_closing_prices(data, ticker)

In [ ]:
import yfinance as yf

# Get the ticker object for NASDAQ Composite Index
nasdaq_ticker = yf.Ticker("^IXIC")

# Get the list of tickers
nasdaq_symbols_list = nasdaq_ticker.tickers['^IXIC'].history().index.get_level_values(1).unique().tolist()

# Print the list of tickers
print(nasdaq_symbols_list)


In [ ]:
db_utils.create_db()

sec_df = df = pd.read_excel('c:\\Users\\somma\\Documents\\GIT\\pythonForFinance\\\\database\\wilshire_5000_stocks.xlsx')
db_utils.insert_wilshire_stocks(sec_df)

sector = db_utils.get_wilshire_distinct_sectors()
tickers = db_utils.get_wilshire_ticker()
stockTicker = db_utils.get_ticker_from_stock()
print(len(tickers),len(stockTicker))
unique_elements = [x for x in tickers if x not in stockTicker]
print(len(tickers)-len(stockTicker),len(unique_elements))
stocks_by_sector = db_utils.get_wilshire_ticker_by_sector("Industrials")

In [ ]:

for ticker in unique_elements:
    df = api_yf.getFullDataFromYf(ticker)
    # insert data into sqlLite db
    if df is not None and not df.empty:
        db_utils.execute_values(df, 'stock_data', ticker)

In [ ]:
# get data from yf
ticker = 'GOOG'

data = api_yf.getFullDataFromYf(ticker)
# insert data into sqlLite db
db_utils.execute_values(data, 'stock_data', ticker)